<a href="https://colab.research.google.com/github/VindhyaSRajan/Guided-Research/blob/master/LSTM_Trainer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!pip install nltk

In [0]:
#installing all the libraries needed for the task
import pandas as pd
import json
import nltk
from pandas.io.json import json_normalize
nltk.download('stopwords')
nltk.download('punkt')
import re  
from nltk.corpus import stopwords
stops1 = set(stopwords.words("english"))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [0]:
! pip install gensim
! pip install seaborn
! pip install elasticsearch
! pip install esengine
! pip install ipdb

     |████████████████████████████████| 92kB 3.8MB/s 
ERROR: google-colab 1.0.0 has requirement six~=1.12.0, but you'll have six 1.10.0 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: six 1.12.0
    Uninstalling six-1.12.0:
      Successfully uninstalled six-1.12.0


  Created wheel for ipdb: filename=ipdb-0.12.3-cp36-none-any.whl size=9223 sha256=959bab38e2ba0c1f30a22598e96c06d0c71d3fe9b3a8587f87492e9e2ed6e41b
  Stored in directory: /root/.cache/pip/wheels/57/43/c5/614153606de8f5e358e266723f53254e70752f4ffc8c85ec63
Successfully built ipdb


In [0]:
%pwd

'/content'

In [0]:
from preprocess import *
from sub_find import *
from true_subs import *
from constants import *
# from graph_evaluation import *
from Vocabulary import Vocab
import pickle

In [0]:
from recipe import Recipe
def load_data_from_file(filename=None, ing_file_name=None, local=True, save=False):
    return load_data(file_name=filename, ing_file_name=ing_file_name, local=local, save=save)

In [0]:
import os
#os.chdir('raw_data')
recipes = load_data_from_file("/content/drive/My Drive/Dataa/500ksample", ing_file_name = "dbpedia_ingredients.txt",local=True, save=False )
clean_recipes = [x.proccessed_ing_list for x in recipes]
print(type(recipes))

<class 'list'>


In [0]:
recipes_1 = load_data_from_file("/content/drive/My Drive/Dataa/500ksample2", "dbpedia_ingredients.txt")
recipes.extend(recipes_1)
sample500k2 = [x.proccessed_ing_list for x in recipes]
clean_recipes.extend(sample500k2) 

In [0]:
#including the following to pickle the 1 million file. It has been split into 3 files with 500k samples in the first two using "split -l 500000 layer1.json"
recipes_2 = load_data_from_file("/content/500ksample3", "dbpedia_ingredients.txt")
recipes.extend(recipes_2)
sample500k3 = [x.proccessed_ing_list for x in recipes]
clean_recipes.extend(sample500k3)
print('The type of data in clean_recipes is ',type(clean_recipes))

The type of data in clean_recipes is  <class 'list'>


In [0]:
print('The final length of the list for the entire dataset is ', len(clean_recipes))
print('The final length of the  recipe list for the entire dataset is ', len(recipes))

The final length of the list for the entire dataset is  499999
The final length of the  recipe list for the entire dataset is  499999


In [0]:
# currently looking only on the second set of elements
#TO-DO : Currently doing it for 100k samples, will later move this to 1M recipes : DONE
total_number_of_recipes = len(recipes)
print(total_number_of_recipes)

499999


In [0]:
recipes #Contains Recipe name and it's ingredients

[Worlds Best Mac and Cheese
 ['penne', 'cheese', 'chili powder', 'butter', 'flour', 'milk', 'hard cheese', 'garlic powder'],
 Dilly Macaroni Salad Recipe
 ['macaroni', 'cheese', 'celery', 'green', 'pimento', 'salad dressing', 'vinegar'],
 Gazpacho
 ['tomatoes', 'salt', 'red onion', 'bell pepper', 'cucumber', 'olive oil', 'basil'],
 Crunchy Onion Potato Bake
 ['milk', 'water', 'butter', 'mashed potato', 'corn', 'cheddar cheese', 'fried onion'],
 Cool 'n Easy Creamy Watermelon Pie
 ['water', 'boiling water', 'cool whip', 'seed', 'graham cracker crust'],
 Easy Tropical Beef Skillet
 ['coconut', 'ground beef', 'garlic', 'black pepper', 'lemon juice', 'soy sauce', 'starch', 'pineapple', 'mandarin orange', 'cashew nut'],
 Kombu Tea Grilled Chicken Thigh
 ['chicken', 'tea', 'white pepper'],
 Strawberry Rhubarb Dump Cake
 ['rhubarb', 'granulated sugar', 'berry', 'cake', 'water', 'butter'],
 Yogurt Parfaits
 ['vanilla', 'ice', 'granola'],
 Zucchini Nut Bread
 ['flour', 'cinnamon', 'baking soda'

In [0]:
clean_recipes #Contains only ingredients

In [0]:
titles_present = set()
for reduced_recipe_object in recipes:
  titles_present.add(str(reduced_recipe_object.id))

In [0]:
len(titles_present)

420207

In [0]:
titles_present #Contains only the titles of the recipes i.e Recipe Names 

In [0]:
map_list = ['soup',
    'sandwich',
    'cocktail',
    'pudding',
    'wraps',
    'doughnut',
    'bread',
    'smoothie',
    'sorbet',
    'stew',
    'cake',
    'condiments',
    'taco',
    'salad',
    'noodles',
    'hummus',
    'burger',
    'steak',
    'pie',
    'custard',
    'chips',
    'casserole',
    'pasta',
    'pizza',
    'omelette',
    'chicken',
    'beef',
    'spring roll',
    'pancake',
    'pastry',
    'salmon',
    'spread']

In [0]:
set(titles_present) & set(map_list)

{'cake', 'pancake', 'pizza', 'smoothie'}

In [0]:
#Circumvent the problem of the 1M datset. Class 38, which is Spring Roll doesn't exist. Remove it.
if 'Spread' in titles_present:
   print('Present')
else:
  print('Not present')

Not present


In [0]:
vindhya_map = {
   'cocktail' : ['cocktail'],
     'wraps' : ['wraps'],
    'doughnut' : ['doughnut'],
    'sorbet' : ['sorbet'],
    'cake' :["muffin",],
    'taco' : ["quessidila"],
    'steak' : ['steak'],
    'casserole' : ['casserole'],
     'spread' : ['spread']
    
}    #These are not present by the above logic and they have been checked in Upper and Lower case

In [0]:
vars(recipes[0])

{'id': 'Worlds Best Mac and Cheese',
 'ingredients_text': '6 ounces penne 2 cups beechers flagship cheese sauce (recipe follows) 1 ounce cheddar, grated (1/4 cup) 1 ounce gruyere cheese, grated (1/4 cup) 1/4 to 1/2 teaspoon chipotle chili powder (see note) 1/4 cup (1/2 stick) unsalted butter 1/3 cup all-purpose flour 3 cups milk 14 ounces semihard cheese (page 23), grated (about 3 1/2 cups) 2 ounces semisoft cheese (page 23), grated (1/2 cup) 1/2 teaspoon kosher salt 1/4 to 1/2 teaspoon chipotle chili powder 1/8 teaspoon garlic powder (makes about 4 cups)',
 'instructions_text': 'preheat the oven to 350 f. butter or oil an 8-inch baking dish. cook the penne 2 minutes less than package directions. (it will finish cooking in the oven.) rinse the pasta in cold water and set aside. combine the cooked pasta and the sauce in a medium bowl and mix carefully but thoroughly. scrape the pasta into the prepared baking dish. sprinkle the top with the cheeses and then the chili powder. bake, uncove

In [0]:
final_dataset = [(x.proccessed_ing_list, x.id) for x in recipes]
print(type(final_dataset))
final_dataset[0] #Final Dataset contains only the processed ingredient list

<class 'list'>


(['penne',
  'cheese',
  'chili powder',
  'butter',
  'flour',
  'milk',
  'hard cheese',
  'garlic powder'],
 'Worlds Best Mac and Cheese')

In [0]:
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()
# type(titles_present)

In [0]:
def stem_title(word):
    return stemmer.stem(word)

In [0]:
# an example
# stem_title('Carsons Chicken Wellington W/ Exotic Mushroom Sherry Sauce')
stem_title('Blueberry Burger')

'blueberry burg'

In [0]:
updated_title = [stem_title(title) for title in titles_present]

In [0]:
updated_title

In [0]:
#New Cell. Not stemming, only converting to lower case
updated_title = [title.lower() for title in titles_present]

In [0]:
type(updated_title)

list

In [0]:
set(updated_title) & set(map_list) #For the 1M dataset, cross check what we have in common with the stemmed title to the original map

{'bread',
 'burger',
 'cake',
 'chicken',
 'custard',
 'doughnut',
 'pasta',
 'pie',
 'pizza',
 'salad',
 'salmon',
 'sandwich',
 'soup',
 'spread',
 'steak',
 'stew',
 'taco'}

In [0]:
with open('recipe_titles_stemmed_alles.txt', 'w') as file:
     for item in updated_title:
         file.write("%s\n" % item)

In [0]:
my_list = 'cake,pasta,custard,rolls,stew,casserole,sauce,soup,burgers,pizza,salad,muffins,steaks,fish,ham'.split(",") # Do Not Run

In [0]:
vindhya_list = ['Cake', 'Salad', 'Noodles', 'Hummus', 'Burger', 'Steak', 'Pie', 'Chicken', 'Custard', 'Chips',
                'Casserole', 'Pasta', 'Pizza', 'Omelette', 'soup', 'sauce'] #Do Not Run

In [0]:
{'bread',
 'burger',
 'cake',
 'chicken',
 'custard',
 'doughnut',
 'pasta',
 'pie',
 'pizza',
 'salad',
 'salmon',
 'sandwich',
 'soup',
 'spread',
 'steak',
 'stew',
 'taco'}

In [0]:
vindhya_map = {
    'soup' : ['soup'],
    'sandwich' : ['sandwich'],
    'cocktail' : ['cocktail'],
    'pudding' : ['pudding'],
    'wraps' : ['wraps'],
    'doughnut' : ['doughnut'],
    'bread' : ['bread'],
    'smoothie' : ['smoothie'],
    'sorbet' : ['sorbet'],
    'stew' : ["daal", "chili", "goulash", "stew"],
    'cake' :["muffin", "cake"],
    'condiments' : ["pickles", "sauce"],
    'taco' : ["quessidila", "fajitas", "taco"],
    'salad' : ["guacamole", "salad"],
    'noodles' : ["noodles"],
    'hummus' : ['hummus'],
    'burger' : ['burger'],
    'steak' : ['steak'],
    'pie' : ['pie'],
    'custard' : ['custard'],
    'chips' : ['chips'],
    'casserole' : ['casserole'],
    'pasta' : ['pasta'],
    'pizza' : ['pizza'],
    'omelette' : ['omelette'],
    'chicken' : ['chicken'],
    'beef' : ['beef'],
    'spring roll' : ['spring roll'],
    'pancake' : ['pancake'],
    'pastry' : ['pastry'],
    'salmon' : ['salmon'],
    'spread' : ['spread']
    
}

In [0]:
vindhya_map = dict((stemmer.stem(k), list(map(stemmer.stem, v))) for k, v in vindhya_map.items())

In [0]:
import itertools
all_recipe_list = list(itertools.chain(*vindhya_map.values()))

In [0]:
# check if the recipe matches
# https://stackoverflow.com/questions/6531482/how-to-check-if-a-string-contains-an-element-from-a-list-in-python
counter = 0
list(itertools.chain(*vindhya_map.values()))
for reduced_recipe_object in recipes:
    # Honestly this is a shitty attempt since I already did the stemming earlier
    # however, I am doing the same again
    counter = counter +  any (stem_title(title) in stem_title(reduced_recipe_object.id) for title in all_recipe_list)
    
counter

219804

In [0]:
# NEW CELL 
#check if the recipe matches
# https://stackoverflow.com/questions/6531482/how-to-check-if-a-string-contains-an-element-from-a-list-in-python
counter = 0
list(itertools.chain(*vindhya_map.values()))
for reduced_recipe_object in recipes:
    # Honestly this is a shitty attempt since I already did the stemming earlier
    # however, I am doing the same again
    #reduced_recipe_object.id gives the titles of all recipes in the 1M dataset
    counter = counter +  any (reduced_recipe_object.id for title in all_recipe_list)
    
counter

1029720

The next 2 cells need not to be run now. The version is updated with the grouped categories.

In [0]:
#Updated the list now. Run this cell only
vindhya_list = ['Cake', 'Salad', 'Noodles', 'Hummus', 'Burger', 'Steak', 'Pie', 'Custard', 'Chips', 'Casserole', 'Pasta', 'Pizza', 'Omelette', 'Soup', 'Sauce','Sandwich','Muffin','Pickles','Cocktail','Pudding','Wraps','Taco','Daal','Guacamole','Doughnut','Bread','Smoothie','Sorbet','Chili','Stew','Chicken','Beef',]

In [0]:
# check if the recipe matches
# https://stackoverflow.com/questions/6531482/how-to-check-if-a-string-contains-an-element-from-a-list-in-python
counter = 0
for reduced_recipe_object in recipes:
    # Honestly this is a shitty attempt since I already did the stemming earlier
    # however, I am doing the same again
    counter = counter +  any (stem_title(title) in stem_title(reduced_recipe_object.id) for title in vindhya_list)
    
counter

41942

In [0]:
percentage_covered = counter / total_number_of_recipes * 100
print(percentage_covered)

43.96088792177584


In [0]:
# so sample 100k2 as we are working on the clean_recipes
final_dataset = [(x.proccessed_ing_list, x.id) for x in recipes]
only_ingredients = [i[0] for i in final_dataset]

In [0]:
type(final_dataset)

list

In [0]:
only_ingredients

In [0]:
type(final_dataset[0]) #should be a tuple

tuple

In [0]:
ingredient_vocabulary = Vocab()
for recipe in only_ingredients:
    for ingredients in recipe:
        ingredient_vocabulary.add_word(ingredients)

In [0]:
# Let us see the vocabulary size once
ingredient_vocabulary.max_idx

1267

In [0]:
ingredient_vocabulary.get_word(1025)

"cow's milk"

In [0]:
import pandas as pd
df = pd.DataFrame(final_dataset, columns =['Ingredients','Recipe_id'])
print(df.shape)

(499999, 2)


In [0]:
df.head()

,Ingredients,Recipe_id
0,"[penne, cheese, chili powder, butter, flour, m...",Worlds Best Mac and Cheese
1,"[macaroni, cheese, celery, green, pimento, sal...",Dilly Macaroni Salad Recipe
2,"[tomatoes, salt, red onion, bell pepper, cucum...",Gazpacho
3,"[milk, water, butter, mashed potato, corn, che...",Crunchy Onion Potato Bake
4,"[water, boiling water, cool whip, seed, graham...",Cool 'n Easy Creamy Watermelon Pie


In [0]:
df_cake = df[df['Recipe_id'].str.contains("Cake")]
df_cake.shape

(19771, 2)

In [0]:
      #Merging the two pandas df to create the one for just 100 samples per category
df2  = pd.read_csv('/content/Food_Categories_100PerCat.csv')
df2.head()
#df[df.Recipe_id == 'Screwdriver Cocktail'] #Just a sanity check that we have all the recipes in both the data frames to join by
df2['Recipe_id'] = df2.Recipe_Title.values
df2.head()
df = df.merge(df2,on = 'Recipe_id')
print(df.shape)
print(df.columns)


(3278, 4)
Index(['Ingredients', 'Recipe_id', 'Recipe_Title', 'Food_Category'], dtype='object')


In [0]:
#Sanity Check to see if all recipe categories are avaialable
print(df.shape)
df['COUNTER'] = 1
grouped_data = df.groupby(['Food_Category'])['COUNTER'].sum()
grouped_data

In [0]:
#Since we have all food categories, let's drop the unecessary columns and have one final df
df = df.drop(columns = ['Recipe_Title', 'Food_Category','COUNTER'])
print(df.columns)
print(df.shape)
print('----After Dropping unwanted columns----')
print(df.columns)
print(df.shape)
df.head()

Index(['Ingredients', 'Recipe_id'], dtype='object')
(3278, 2)
----After Dropping unwanted columns----
Index(['Ingredients', 'Recipe_id'], dtype='object')
(3278, 2)


,Ingredients,Recipe_id
0,"[cream cheese, condensed milk, peanut butter, ...",Peanut Butter Pie
1,"[chocolate, sugar, butter, cream cheese, peanu...",Peanut Butter Pie
2,"[graham cracker, peanut butter, butter, cream ...",Peanut Butter Pie
3,"[graham cracker crust, cream cheese, peanut bu...",Peanut Butter Pie
4,"[cream cheese, peanut butter, sugar, milk, hea...",Peanut Butter Pie


In [0]:
!wget http://nlp.stanford.edu/data/glove.6B.zip

--2020-02-03 10:20:05--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2020-02-03 10:20:05--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2020-02-03 10:20:05--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6

In [0]:
!unzip '/content/glove.6B.zip'

Archive:  /content/glove.6B.zip
  inflating: glove.6B.50d.txt        
  inflating: glove.6B.100d.txt       
  inflating: glove.6B.200d.txt       
  inflating: glove.6B.300d.txt       


In [0]:
from gensim.test.utils import datapath
from gensim.scripts.glove2word2vec import glove2word2vec

glove_file = datapath('/content/glove.6B.100d.txt')
with open('glove.100d.w2v', mode='wb+') as dest_file:
    glove2word2vec(glove_file, dest_file)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
import gensim
pretrained_embedding = gensim.models.KeyedVectors.load_word2vec_format('/content/glove.100d.w2v')

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
# Just a sample check to indicate things are working as expected
pretrained_embedding.vocab['apple'].index

3292

In [0]:
# This is more of a hack
def recipe_vocab_on_list(ingedient_list):
    result_list = []
    for ingedient in ingedient_list:
        # If multi word ingredient, chose the first word
        ingedient = ingedient.split(" ")[0]
        try:
            result_list.append(pretrained_embedding.vocab[ingedient].index)
        except KeyError:
            result_list.append(pretrained_embedding.vocab['unk'].index)
    return result_list

In [0]:
df['Ingredient_Numeric'] = df.Ingredients.apply(recipe_vocab_on_list)

In [0]:
df.iloc[5]

Ingredients           [coconut, ground beef, garlic, black pepper, l...
Recipe_id                                    Easy Tropical Beef Skillet
Ingredient_Numeric    [13890, 817, 9005, 521, 9311, 14386, 28710, 22...
Name: 5, dtype: object

In [0]:
%pwd

'/content'

In [0]:
recipe_vocabulary = Vocab()
for recipe in vindhya_map:
    recipe_vocabulary.add_word(stemmer.stem(recipe))

In [0]:
#NewCell : Without Stemming only lower cased
recipe_vocabulary = Vocab()
for recipe in vindhya_map:
    recipe = recipe.lower()
    recipe_vocabulary.add_word(recipe)

In [0]:
recipe_vocabulary

{'<UNK>': 0, 'soup': 1, 'sandwich': 2, 'cocktail': 3, 'pud': 4, 'wrap': 5, 'doughnut': 6, 'bread': 7, 'smoothi': 8, 'sorbet': 9, 'stew': 10, 'cake': 11, 'condiment': 12, 'taco': 13, 'salad': 14, 'noodl': 15, 'hummu': 16, 'burger': 17, 'steak': 18, 'pie': 19, 'custard': 20, 'chip': 21, 'casserol': 22, 'pasta': 23, 'pizza': 24, 'omelett': 25, 'chicken': 26, 'beef': 27, 'spring rol': 28, 'pancak': 29, 'pastri': 30, 'salmon': 31, 'spread': 32}

In [0]:
# Just a sample check to indicate things are working as expected
pretrained_embedding.vocab['salmon'].index 

7411

In [0]:
# This is more of a hack
# def recipe_vocab_on_list(ingedient_list):
#     result_list = []
#     for ingedient in ingedient_list:
#         # If multi word ingredient, chose the first word
#         ingedient = ingedient.split(" ")[0]
#         try:
#             result_list.append(pretrained_embedding.vocab[ingedient].index)
#         except KeyError:
#             result_list.append(pretrained_embedding.vocab['unk'].index)
#     return result_list

In [0]:
# df['Ingredient_Numeric'] = df.Ingredients.apply(recipe_vocab_on_list)

In [0]:
# df.iloc[5]

In [0]:
len(recipe_vocabulary.word_2_idx)

33

In [0]:
recipe_vocabulary.get_all_words_in_vocab_mit_map(stemmer.stem('The Spiced Cake Project'),
                                                vindhya_map)
# recipe_vocabulary.get_idx('cake')

(True, [11])

In [0]:
#New Cell : No Stemming, only lower cased
recipe_vocabulary.get_all_words_in_vocab_mit_map(('The Spiced Cake Project').lower(),
                                                vindhya_map)
# recipe_vocabulary.get_idx('cake')

(True, [11])

In [0]:
# Uptil this point we were simply returning the first match in the recipe vocabulary. 
# This can be modified and we look for all the matches of possible recipe
# Since the list is sorted specific to our rule, we would go ahead and take the minimum value from all
def get_recipe_idx(recipe):
    recipe_stemmed = stemmer.stem(recipe)
    lookup_result = recipe_vocabulary.get_all_words_in_vocab(recipe_stemmed)
    #lookup_result = Vocab.get_all_words_in_vocab(recipe_stemmed)
    if lookup_result[0]:
        print(lookup_result[1])
        return min(lookup_result[1])
    # No match so 0 returned
    return  0

In [0]:
#New Cell : No stemming, only lower cased
def get_recipe_idx(recipe):
    recipe_lower = recipe.lower()
    lookup_result = recipe_vocabulary.get_all_words_in_vocab(recipe_lower)
    #lookup_result = Vocab.get_all_words_in_vocab(recipe_stemmed)
    if lookup_result[0]:
        print(lookup_result[1])
        return min(lookup_result[1])
    # No match so 0 returned
    return  0

In [0]:
# recipe_vocabulary.check_any_word_in_vocab(stemmer.stem('The Spiced Cake Project'))
# recipe_vocabulary.get_idx('cake')

(True, 'cake')

In [0]:
# def get_recipe_idx(recipe):
#     recipe_stemmed = stemmer.stem(recipe)
#     lookup_result = recipe_vocabulary.check_any_word_in_vocab(recipe_stemmed)
#     if lookup_result[0]:
#         return recipe_vocabulary.get_idx(lookup_result[1])
#     return 0

In [0]:
get_recipe_idx('The Spiced Cake Project')
#get_recipe_idx('Sugarless Applesauce Cake')

[11]


11

In [0]:
df['Recipe_id_numeric'] = df.Recipe_id.apply(get_recipe_idx)

[14]
[27]
[26]
[7]
[31, 14]
[22]
[18, 5]
[1]
[19, 8]
[2]
[7]
[22]
[14]
[2]
[31, 1]
[26, 23]
[19]
[1]
[27]
[14]
[7]
[1]
[14]
[14]
[27]
[26]
[25]
[7]
[26]
[14]
[26]
[1]
[26]
[6]
[14]
[14]
[23]
[26]
[26]
[15]
[24]
[11]
[26]
[11]
[1]
[26, 1]
[1]
[26]
[32]
[26]
[8]
[18]
[23]
[14]
[11]
[26]
[25]
[7]
[15]
[26]
[24]
[26]
[10]
[26]
[14]
[1]
[26]
[26]
[14]
[26, 10]
[14]
[1]
[31]
[26]
[13, 1]
[11]
[26]
[22]
[2]
[26]
[14]
[2]
[24]
[26]
[10]
[24]
[26]
[26]
[27]
[26, 13]
[31]
[14]
[22]
[27, 18, 4]
[29]
[1]
[27]
[7]
[26]
[7]
[26]
[26]
[22]
[26]
[26, 22]
[23]
[26]
[26]
[26]
[7]
[14]
[2]
[11]
[18]
[1]
[26, 13, 14]
[22]
[23]
[20]
[7]
[26]
[1]
[1]
[7]
[31]
[27]
[27, 1]
[1]
[7]
[18]
[14]
[2]
[26]
[22]
[2]
[7]
[7]
[1]
[7, 4]
[14]
[23]
[14]
[21]
[23]
[26]
[23]
[14]
[21]
[11]
[18]
[7]
[26]
[14]
[18]
[26, 14]
[26]
[4]
[1]
[21]
[7]
[23, 14]
[29]
[24]
[26]
[26]
[14]
[14]
[26, 14]
[22]
[14]
[7]
[26]
[13, 14]
[24]
[26]
[1]
[14]
[26]
[8]
[26]
[2]
[26]
[1]
[26, 1]
[27]
[27]
[1]
[23, 23]
[26]
[26, 19]
[32]
[14]
[21]

In [0]:
df.iloc[98]

Ingredients           [acorn, bean, tomatoes, onion, garlic, vegetab...
Recipe_id                                Acorn Squash & Black Bean Soup
Ingredient_Numeric    [25260, 11239, 9077, 10952, 9005, 9144, 6952, ...
Recipe_id_numeric                                                     1
Name: 98, dtype: object

In [0]:
df.Recipe_id_numeric.value_counts()#Just Checking here that all categories have some data in them

0     333308
26     33179
14     28911
1      18065
7      11405
22      7222
27      7124
23      6206
2       5450
24      5047
11      4840
31      4297
18      4181
19      4132
21      4116
10      4084
4       2985
13      2480
8       2164
29      2094
3       1503
32      1406
5       1337
15      1178
20       877
9        618
16       508
17       421
30       332
6        321
25       207
12         1
Name: Recipe_id_numeric, dtype: int64

In [0]:
len(df)

499999

In [0]:
# df = df.drop(columns= ['COUNTER']) #dropping the COUNTER column because it did the task above
df.columns

Index(['Ingredients', 'Recipe_id', 'Ingredient_Numeric', 'Recipe_id_numeric'], dtype='object')

In [0]:
recipe_vocabulary

{'<UNK>': 0, 'soup': 1, 'sandwich': 2, 'cocktail': 3, 'pud': 4, 'wrap': 5, 'doughnut': 6, 'bread': 7, 'smoothi': 8, 'sorbet': 9, 'stew': 10, 'cake': 11, 'condiment': 12, 'taco': 13, 'salad': 14, 'noodl': 15, 'hummu': 16, 'burger': 17, 'steak': 18, 'pie': 19, 'custard': 20, 'chip': 21, 'casserol': 22, 'pasta': 23, 'pizza': 24, 'omelett': 25, 'chicken': 26, 'beef': 27, 'spring rol': 28, 'pancak': 29, 'pastri': 30, 'salmon': 31, 'spread': 32}

In [0]:
ingredient_vocabulary

{'<UNK>': 0, 'penne': 1, 'cheese': 2, 'chili powder': 3, 'butter': 4, 'flour': 5, 'milk': 6, 'hard cheese': 7, 'garlic powder': 8, 'macaroni': 9, 'celery': 10, 'green': 11, 'pimento': 12, 'salad dressing': 13, 'vinegar': 14, 'tomatoes': 15, 'salt': 16, 'red onion': 17, 'bell pepper': 18, 'cucumber': 19, 'olive oil': 20, 'basil': 21, 'water': 22, 'mashed potato': 23, 'corn': 24, 'cheddar cheese': 25, 'fried onion': 26, 'boiling water': 27, 'cool whip': 28, 'seed': 29, 'graham cracker crust': 30, 'coconut': 31, 'ground beef': 32, 'garlic': 33, 'black pepper': 34, 'lemon juice': 35, 'soy sauce': 36, 'starch': 37, 'pineapple': 38, 'mandarin orange': 39, 'cashew nut': 40, 'chicken': 41, 'tea': 42, 'white pepper': 43, 'rhubarb': 44, 'granulated sugar': 45, 'berry': 46, 'cake': 47, 'vanilla': 48, 'ice': 49, 'granola': 50, 'cinnamon': 51, 'baking soda': 52, 'baking powder': 53, 'egg': 54, 'sugar': 55, 'vegetable oil': 56, 'zucchini': 57, 'walnut': 58, 'onion': 59, 'red pepper': 60, 'salmon': 6

In [0]:
with open('/content/drive/My Drive/vocab_combined_500k.pkl','wb') as file:
    pickle.dump(recipe_vocabulary, file)
    pickle.dump(ingredient_vocabulary, file)

In [0]:
def check_empty(x):
    return len(x) == 0

In [0]:
processed_df = df.drop(df[df.Ingredient_Numeric.apply(check_empty)].index)

In [0]:
assert sum(processed_df.Ingredient_Numeric.apply(check_empty)) == 0

In [0]:
%pwd

'/content'

In [0]:
# I am just saving the file here for reference later
pd.to_pickle(processed_df, '/content/drive/My Drive/processed_df_500k.pkl')

In [0]:
from sklearn.model_selection import train_test_split
df_train, df_test = train_test_split(processed_df, test_size = 0.2, random_state = 0)

In [0]:

# I am assuming that we are able to get the categories here
# Now this should become a LSTM based model which will try and do binary prediction
import torch
import torch.nn as nn
import torch.utils.data as data
import torch.utils.data.dataloader as dataloader
from torch.nn.utils.rnn import pad_sequence

In [0]:

# Some constants
batch_size = 5

In [0]:
# https://github.com/yunjey/seq2seq-dataloader/blob/master/data_loader.py
def collate_fn(data):
    """Creates mini-batch tensors from the list of tuples (src_seq, trg_seq).
    We should build a custom collate_fn rather than using default collate_fn,
    because merging sequences (including padding) is not supported in default.
    Seqeuences are padded to the maximum length of mini-batch sequences (dynamic padding).
    Args:
        data: list of tuple (src_seq, trg_seq).
            - src_seq: torch tensor of shape (?); variable length.
            - trg_seq: torch tensor of shape (?); variable length.
    Returns:
        src_seqs: torch tensor of shape (batch_size, padded_length).
        src_lengths: list of length (batch_size); valid length for each padded source sequence.
        trg_seqs: torch tensor of shape (batch_size, padded_length).
        trg_lengths: list of length (batch_size); valid length for each padded target sequence.
    """
    def merge(sequences):
        lengths = [len(seq) for seq in sequences]
        padded_seqs = torch.zeros(len(sequences), max(lengths)).long()
        for i, seq in enumerate(sequences):
            end = lengths[i]
            padded_seqs[i, :end] = seq[:end]
        return padded_seqs, lengths

    # sort a list by sequence length (descending order) to use pack_padded_sequence
#     print(data[0]) # list of tuples
    data.sort(key=lambda x: len(x[0]), reverse=True)

    # seperate source and target sequences
    src_seqs, trg_seqs = zip(*data)

    # merge sequences (from tuple of 1D tensor to 2D tensor)
    src_seqs, src_lengths = merge(src_seqs)
    # target sequence for us is a single tensor so we do not need to 
    # merge it
    #trg_seqs, trg_lengths = merge(trg_seqs)
    trg_seqs = torch.as_tensor(trg_seqs)
    return src_seqs, src_lengths, trg_seqs #, trg_lengths

In [0]:
class RecipeData(data.Dataset):
    
    def __init__(self, df):
        super(RecipeData, self).__init__()
        self.df = df
    
    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        X = torch.as_tensor(self.df.Ingredient_Numeric.iloc[idx])
        y = torch.as_tensor(self.df.Recipe_id_numeric.iloc[idx])
        return X,y

In [0]:
train_dataset = RecipeData(df_train)
test_dataset = RecipeData(df_test)
train_data_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                              batch_size=batch_size,
                                              shuffle=True,
                                              collate_fn=collate_fn,
                                               drop_last=True) # Done for cases when num_samples not exact multiple
test_data_loader = torch.utils.data.DataLoader(dataset=test_dataset, # of the batch_size
                                              batch_size=batch_size,
                                              shuffle=True,
                                              collate_fn=collate_fn,
                                              drop_last=True)

In [0]:
df.Recipe_id_numeric[4]

In [0]:
train_dataset[0]

In [0]:
for (X,X_len,y) in train_data_loader:
    print(X)
    print(X_len)
    print(y)
    break

In [0]:
X

In [0]:
class RecipePredictor(nn.Module):
    
    def __init__(self, vocab_size, hidden_dim, embedding_dim, batch_size, output_dim):
        super(RecipePredictor, self).__init__()
        self.vocab_size = vocab_size
        self.output_dim = output_dim
        self.hidden_dim = hidden_dim
        self.embedding_dim = embedding_dim
        self.embedding = nn.Embedding(self.vocab_size, self.embedding_dim)
        self.lstm = nn.LSTM(input_size=self.embedding_dim, hidden_size=self.hidden_dim)
        self.batch_size = batch_size
        self.predictor = nn.Linear(self.hidden_dim, self.output_dim)
        self.non_linearity = nn.ReLU()
        self.init_hidden() # TODO:  This should happen at the beginning of each epoch
        
    def init_hidden(self):
        self.h_n = torch.randn(1, self.batch_size, self.hidden_dim)
        self.c_n = torch.randn(1, self.batch_size, self.hidden_dim)
        
    
    def forward(self, input_sequence, max_len):
#         print(input_sequence)
#         print(max_len)
        embedded = self.embedding(input_sequence)
        packed = nn.utils.rnn.pack_padded_sequence(embedded, max_len, batch_first=True)
        outputs, (self.c_n, self.h_n) = self.lstm(packed, (self.c_n, self.h_n))
        # Unpack padding
        """
            Honestly, I do not know if at this point, I need the output. I would rather prefer to work with the
            self.h_n cell and so will not `pad_padded_sequence`
        """
        #outputs, _ = nn.utils.rnn.pad_packed_sequence(outputs, batch_first=True)
        # on which output should the prediction be done?
        # self.h_n =  num_layers, batch_size, hidden_dim
        batch_size = self.h_n.shape[1]
        output_predicted = self.predictor(self.h_n.reshape(batch_size, -1))
        return output_predicted

In [0]:
vocab_size = ingredient_vocabulary.max_idx + 1

In [0]:
num_recipes = recipe_vocabulary.max_idx+1
model = RecipePredictor(vocab_size=vocab_size, hidden_dim=512, embedding_dim=300, batch_size=batch_size,
                       output_dim=num_recipes)

In [0]:
model(X, X_len)

In [0]:
lr = 1e-3
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [0]:
num_epoch = 2

In [0]:
# Moving things to GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

In [0]:
model.to(device)

In [0]:
def put_elements_to_device(a,b,c,device):
    return a.to(device), b.to(device), c.to(device)

In [0]:

# !pip install -q tb-nightly

# Load the TensorBoard notebook extension
%load_ext tensorboard

In [0]:
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()

In [0]:
print(len(train_data_loader) * batch_size)
print(len(df_train))
# Not same since we are dropping some terms which do not match up
N_train = len(train_data_loader) * batch_size
N_test = len(test_data_loader) * batch_size

In [0]:
for epoch in range(num_epoch):
    running_loss = 0
    model.init_hidden()
    for idx,(X,X_len,y) in enumerate(train_data_loader):
        model.zero_grad()
        X, X_len, y = put_elements_to_device(a=X, b=torch.tensor(X_len), c=y, device=device)
        prediction = model(X, X_len)
        loss = criterion(prediction, y)
        loss.backward(retain_graph=True)
        optimizer.step()
        running_loss += loss.item()
        if idx % 100 == 0:
            print("epoch {} loss {}".format(epoch, running_loss))
    writer.add_scalar('Loss/train', running_loss/N_train, epoch)
    # validation set is still left to create
    # Now to test the validation set
    correct = 0
    total = 0
    test_loss = 0
    with torch.no_grad():
        for idx,(X,X_len,y) in enumerate(test_data_loader):
            X, X_len, y = put_elements_to_device(a=X, b=torch.tensor(X_len), c=y, device=device)
            outputs = model(X,X_len)
            loss = criterion(outputs, y)
            test_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += X.size(0)
            correct += (predicted == y).sum().item()
    writer.add_scalar('Loss/train', test_loss/N_test, epoch)
    print('Accuracy of the network on the test samples: %d %%' % (
        100 * correct / total))

In [0]:
%load_ext tensorboard
%tensorboard --logdir=runs

In [0]:
torch.__version__

In [0]:
vindhya_data_df = pd.read_csv('./recipe_title_foodCategory.csv')

In [0]:
list(set(vindhya_data_df.Food_Category))

In [0]:
a = [1,2,3,4,5]
b = ['a','b','c','d','e']
c = ['dil','me','dard','e','disco']
sallu_bhoi = zip(a,b,c)

In [0]:
a,b,c = zip(*sallu_bhoi)

In [0]:
np.asarray(a)